# Load fritz.science source file

In [1]:
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from astropy.table import Table, join
from astropy.coordinates import SkyCoord
from astropy import units as u

from astroquery.gaia import Gaia
from astroquery.vizier import Vizier
#tables = Gaia.load_tables(only_names=True)
#for table in tables:
#    print(table.name)

In [2]:
sources_df = pd.read_csv("data/sources.csv")
sources_df


,id,ra [deg],dec [deg],redshift,classification,probability,annotation origin,annotation origin key-value pair count,annotation key,annotation value,groups,Saved at,Alias,Origin,TNS
0,ZTF25aceekci,179.440246,55.507917,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Fast Transient GCN,2025-11-13T17:42:42,NaN,Kowalski,NaN
1,ZTF25aceemtn,183.287855,62.905781,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Fast Transient GCN,2025-11-13T17:41:57,NaN,Kowalski,NaN
2,ZTF25aceekrw,180.737327,48.352361,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Fast Transient GCN,2025-11-13T17:40:14,NaN,Kowalski,NaN
3,ZTF25aceelbj,169.945969,40.897420,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Fast Transient GCN,2025-11-13T17:38:54,NaN,Kowalski,NaN
4,ZTF25acdeiid,37.873606,7.791892,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Fast Transient GCN,2025-11-11T18:49:20,NaN,Kowalski,AT 2025acwj
5,ZTF25acdhetm,142.885962,59.376197,NaN,NaN,NaN,ls_dr9-9907739645379578;ps1.dr2-17925142886010...,7;8,ra;dec;type;flux_z;z_phot_l95;z_phot_std;z_pho...,142.88595691231106;59.376205672872885;PSF;0.89...,Fast Transient GCN,2025-11-10T17:26:02,NaN,Kowalski,AT 2025acvr
6,ZTF25acdabfa,11.861833,-6.805553,0.056182,NaN,NaN,NaN,NaN,NaN,NaN,Fast Transient GCN,2025-11-10T17:20:36,NaN,Kowalski,AT 2025acuu
7,ZTF25acczwqv,356.601029,19.681700,NaN,NaN,NaN,ls_dr9-9906628824335124,7,ra;dec;type;flux_z;z_phot_l95;z_phot_std;z_pho...,356.60099035254075;19.68189690642853;REX;11.77...,Fast Transient GCN,2025-11-10T16:32:27,NaN,Kowalski,AT 2025acwk
8,ZTF25acceztk,19.383869,16.246087,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Fast Transient GCN,2025-11-10T16:32:27,NaN,Kowalski,AT 2025acre
9,ZTF25accgmpa,153.461746,39.986779,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Fast Transient GCN,2025-11-10T16:32:20,NaN,Kowalski,AT 2025acry


In [3]:
# Function to unpack the columns with specfic data listed.

def unpack_annotations(row):
    # If no annotation for this row → return empty
    if pd.isna(row["annotation origin"]):
        return pd.Series({}, dtype=object)

    # Parse each field as semicolon-separated lists
    origins = str(row["annotation origin"]).split(";")
    counts  = [int(x) for x in str(row["annotation origin key-value pair count"]).split(";")]
    keys    = str(row["annotation key"]).split(";")
    vals    = str(row["annotation value"]).split(";")

    out = {}
    idx = 0

    # For each annotation origin block
    for origin, n in zip(origins, counts):
        these_keys = keys[idx:idx+n]
        these_vals = vals[idx:idx+n]

        for k, v in zip(these_keys, these_vals):
            colname = f"{origin}.{k}"
            out[colname] = v

        idx += n

    return pd.Series(out, dtype=object)


In [4]:
# Execute the function and view the results.

sources_ann_df = sources_df.apply(unpack_annotations, axis=1)
full_sources_df = pd.concat([sources_df, sources_ann_df], axis=1)
full_sources_df


,id,ra [deg],dec [deg],redshift,classification,probability,annotation origin,annotation origin key-value pair count,annotation key,annotation value,...,ls_dr9-9907739645379578.z_phot_median,ls_dr9-9907739645379578.z_phot_std,ps1.dr2-179251428860102091.dec,ps1.dr2-179251428860102091.distance_arcsec,ps1.dr2-179251428860102091.gMeanPSFMag,ps1.dr2-179251428860102091.iMeanPSFMag,ps1.dr2-179251428860102091.rMeanPSFMag,ps1.dr2-179251428860102091.ra,ps1.dr2-179251428860102091.yMeanPSFMag,ps1.dr2-179251428860102091.zMeanPSFMag
0,ZTF25aceekci,179.440246,55.507917,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ZTF25aceemtn,183.287855,62.905781,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,ZTF25aceekrw,180.737327,48.352361,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,ZTF25aceelbj,169.945969,40.897420,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,ZTF25acdeiid,37.873606,7.791892,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,ZTF25acdhetm,142.885962,59.376197,NaN,NaN,NaN,ls_dr9-9907739645379578;ps1.dr2-17925142886010...,7;8,ra;dec;type;flux_z;z_phot_l95;z_phot_std;z_pho...,142.88595691231106;59.376205672872885;PSF;0.89...,...,0.442666,0.239618,59.37618167452314,0.08417874456115809,21.92259979248047,-999,21.799800872802734,142.88599609797456,-999,20.976600646972656
6,ZTF25acdabfa,11.861833,-6.805553,0.056182,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,ZTF25acczwqv,356.601029,19.681700,NaN,NaN,NaN,ls_dr9-9906628824335124,7,ra;dec;type;flux_z;z_phot_l95;z_phot_std;z_pho...,356.60099035254075;19.68189690642853;REX;11.77...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,ZTF25acceztk,19.383869,16.246087,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,ZTF25accgmpa,153.461746,39.986779,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
